<a href="https://colab.research.google.com/github/yuu067/MIA-IABD-2425/blob/main/UD03/notebooks/EX3.-analisi_sentiment_imdb_ES.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ejercicios UD03_03

# Análisis de sentimientos con Distilbert

En la práctica [Analisis de sentimientos con DistilBERT](https://colab.research.google.com/github/martinezpenya/MIA-IABD-2425/blob/main/UD03/notebooks/3.-models_llenguatge_ES.ipynb) hemos entrenado a Distilbert para clasificar los sentimientos en los textos, utilizando una base de datos de Tweets.

En este ejercicio, queremos usar Distilbert para clasificar las reseñas de películas y el mismo entrenamiento no nos sirve. Por esta razón, utilizaremos el aprendizaje de transferencia para adaptar el modelo a la nueva tarea, utilizando el `DataSet` de las reseñas de películas IMDB, disponibles en Huggingface como un `imdb`.

## Objetivos de la práctica

* Reproduce el proceso visto en la práctica [Analisis de sentimientos con DistilBERT](https://colab.research.google.com/github/martinezpenya/MIA-IABD-2425/blob/main/UD03/notebooks/3.-models_llenguatge_ES.ipynb) para entrenar a Distilbert para clasificar los sentimientos en las reseñas de películas.
* Deberá entrenar el modelo (Distilbert u otros transformadores) con el `dataset` de las reseñas de películas IMDB.
* Deberá analizar las predicciones del modelo y compararlo con el modelo entrenado en la práctica [Analisis de sentimientos con DistilBERT](https://colab.research.google.com/github/martinezpenya/MIA-IABD-2425/blob/main/UD03/notebooks/3.-models_llenguatge_ES.ipynb).
* Utilitza un `pipeline` de transformers para hacer predicciones [`zero-shot`](https://huggingface.co/transformers/main_classes/pipelines.html#transformers.ZeroShotClassificationPipeline) sobre reseñas de películas. Comparar los resultados con los del modelo entrenado con el `dataset` de IMDB.
* Escribe una celda con tus comentarios y/o conclusiones finales comparando las dos alternativas realizadas en el ejercicio.

In [29]:
import os
os.environ["WANDB_DISABLED"] = "true"
%pip install -U transformers datasets evaluate accelerate scikit-learn accuracy

In [30]:
import datasets

# Cargamos el Dataset
dataset = datasets.load_dataset('imdb')

# Mostramos los datos de ejemplo

dataset['train'][0]

{'text': 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far be

In [31]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

In [32]:
# importamos el tokenizador de DistilBERT
from transformers import AutoTokenizer

# Cargamos el tokenizador
tokenizer = AutoTokenizer.from_pretrained('distilbert/distilbert-base-uncased')

# Mostramos un ejemplo de tokenización
tokenizer.tokenize('FC Barcelona is fucked this year')

['fc', 'barcelona', 'is', 'fucked', 'this', 'year']

In [33]:
# Definimos una función para preprocesar el texto.
# Truncamos los textos para asegurarnos de que no excedan el máximo tamaño de entrada deDistilBert

def tokenize(examples):
    return tokenizer(examples["text"], padding='max_length', truncation=True)

In [34]:
dades_tokenitzades = dataset.map(tokenize, batched=True)

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [35]:
import evaluate

accuracy = evaluate.load('accuracy')

In [36]:
# Definimos una función para calcular la precisión del modelo

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = predictions.argmax(axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [37]:
id_a_etiqueta = {
    0: "SADNESS",
    1: "JOY",
    2: "LOVE",
    3: "ANGER",
    4: "FEAR",
    5: "SUPRISE"
}

etiqueta_a_id = {
    "SADNESS": 0,
    "JOY": 1,
    "LOVE": 2,
    "ANGER": 3,
    "FEAR": 4,
    "SUPRISE": 5
}

In [38]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(
    'distilbert/distilbert-base-uncased',
    num_labels=len(etiqueta_a_id),
    id2label=id_a_etiqueta,
    label2id=etiqueta_a_id
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [39]:
from transformers import TrainingArguments, Trainer

BATCH_SIZE = 16
NUM_EPOCHS = 1

training_args = TrainingArguments(
    output_dir="test_trainer",
    eval_strategy="epoch",
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=NUM_EPOCHS,
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dades_tokenitzades['train'],
    eval_dataset=dades_tokenitzades['test'],
    compute_metrics=compute_metrics,
)

trainer.train()